In [ ]:
import os, csv
import numpy as np
from tqdm import tqdm
from tensorboard_utils import *
from matplotlib import pyplot as plt

In [ ]:
LOG_DIR = '/home/lhoyer/cnn_robot_localization/training_results/first_stage_2'
OUT_DIR = '/home/lukas/Nextcloud/Studium/Bachelorarbeit/Plots'

plt.style.use('default')
plt.rcParams.update({'font.size': 14})

exp_dict = {
    'F0': 'exp0',
    'F1.1': 'exp1/background_diversity_swarmlab',
    'F1.2': 'exp1/background_diversity_coco',
    # exp3 is correct -> renumbered for paper
    'F2.1': 'exp3/compositor_amount_1000',
    'F2.2': 'exp3/compositor_amount_20000',
    'F3.1': 'exp4/crop_method_automatic',
    'F3.2': 'exp4/crop_method_both',
    'F4.1': 'exp2/first_stage_resolution_200_150',
    'F4.2': 'exp2/first_stage_resolution_800_600',
    #'F5.1': 'exp5/crop_position_4',
    #'F5.2': 'exp5/crop_position_245',
    #'F6.1': 'exp6/lighting_natural',
    #'F6.2': 'exp6/lighting_artificial',
    #'F6.3': 'exp6/lighting_artificial_augmented',
}
exp_dict = OrderedDict(sorted(exp_dict.items()))

working_keys = exp_dict.keys()
for r in ['copter','sphero','youbot','all']:
    ys = []
    for k in working_keys:
        recfiles = []
        for root, subdirs, files in os.walk(LOG_DIR):
            if exp_dict[k] not in root: continue
            if 'eval_' not in root: continue
            for file in files:
                recfiles.append(root+'/'+file)
        recfiles.sort()
        #print('{} has {} files: {}'.format(k,len(recfiles),recfiles))
        
        # Read tfevents
        tensorboards = []
        for file in recfiles:
            if 'tfevents' not in file:
                continue
            try:
                tensorboards.append(readTfEvents(file))
            except Exception as e:
                print('Couldn\'t read {}: {}'.format(file,e))

        
        # Concat tf events
        ys.append([])
        for i,t in enumerate(tensorboards):
            t = filterRange(t)
            try: 
                if r == 'all':
                    metric = 'PascalBoxes_Precision/mAP@0.5IOU'
                else:
                    metric = 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/{}'.format(r)
                y = (1-np.array(t[metric]))*100
                ys[-1].extend(y)
            except KeyError as e:
                print('Couldn\' access tensorboard {}: {}'.format(i,e))
    
    # Draw boxplots
    print('Number of default samples: {}'.format([len(l) for l in ys]))
    bp_dict = plt.boxplot(ys,
                labels=working_keys,
                sym='k+',
                medianprops=dict(color='blue'),
                widths=0.4,
                showfliers=False
               )
    for line in bp_dict['medians']:
        # get position data for median line
        x, y = line.get_xydata()[1] # top of median line
        # overlay median value
        plt.text(x+0.05, y, ('%.1f'%y),
             horizontalalignment='left',
             verticalalignment='center',
             fontsize=8) # draw above, centered

    plt.ylabel('(1 - mAP@0.5IOU) in %')
    plt.xlabel('Experiment', labelpad=15)
    plt.xticks(rotation=90)
    plt.grid()
    plt.gca().set_xlim([0.6,len(working_keys)+0.8])
    #plt.title('First stage performance for robot type: {}'.format(r))
    ax = plt.gca()
    ratio_default=(ax.get_xlim()[1]-ax.get_xlim()[0])/(ax.get_ylim()[1]-ax.get_ylim()[0])
    plt.gca().set_aspect(0.8*ratio_default)
    plt.savefig('{}/first_stage_{}.pdf'.format(OUT_DIR, r), bbox_inches='tight')
    plt.show()

## Presentation Plots 

In [ ]:
LOG_DIR = '/home/lhoyer/cnn_robot_localization/training_results/first_stage_2'
OUT_DIR = '/home/lukas/Nextcloud/Studium/Bachelorarbeit/IEEE SSCI Presentation/Plots'

plt.style.use('default')

exp_dict = {
    #'200x150': 'exp2/first_stage_resolution_200_150',
    #'400x300': 'exp0',
    #'800x600': 'exp2/first_stage_resolution_800_600',
    '1000': 'exp3/compositor_amount_1000',
    '5000': 'exp0',
    '20000': 'exp3/compositor_amount_20000',
}
#exp_dict = OrderedDict((exp_dict.items()))

working_keys = list(reversed(list(exp_dict.keys())))
for r in ['copter','sphero','youbot']:
    ys = []
    for k in working_keys:
        recfiles = []
        for root, subdirs, files in os.walk(LOG_DIR):
            if exp_dict[k] not in root: continue
            if 'eval_' not in root: continue
            for file in files:
                recfiles.append(root+'/'+file)
        recfiles.sort()
        #print('{} has {} files: {}'.format(k,len(recfiles),recfiles))
        
        # Read tfevents
        tensorboards = []
        for file in recfiles:
            if 'tfevents' not in file:
                continue
            try:
                tensorboards.append(readTfEvents(file))
            except Exception as e:
                print('Couldn\'t read {}: {}'.format(file,e))

        
        # Concat tf events
        ys.append([])
        for i,t in enumerate(tensorboards):
            t = filterRange(t)
            try: 
                if r == 'all':
                    metric = 'PascalBoxes_Precision/mAP@0.5IOU'
                else:
                    metric = 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/{}'.format(r)
                y = (1-np.array(t[metric]))*100
                ys[-1].extend(y)
            except KeyError as e:
                print('Couldn\' access tensorboard {}: {}'.format(i,e))
    
    # Draw boxplots
    print('Number of default samples: {}'.format([len(l) for l in ys]))
    bp_dict = plt.boxplot(ys,
                labels=working_keys,
                sym='k+',
                medianprops=dict(color='blue'),
                widths=0.4,
                showfliers=False
               )
    for line in bp_dict['medians']:
        # get position data for median line
        x, y = line.get_xydata()[1] # top of median line
        # overlay median value
        plt.text(x+0.05, y, ('%.1f'%y),
             horizontalalignment='left',
             verticalalignment='center',
             fontsize=8) # draw above, centered

    plt.ylabel('(1 - mAP@0.5IOU) in %')
    plt.xlabel('Generated Images per Type')
    plt.grid()
    plt.gca().set_xlim([0.5,len(working_keys)+0.8])
    plt.title('{}'.format(r))
    ax = plt.gca()
    ratio_default=(ax.get_xlim()[1]-ax.get_xlim()[0])/(ax.get_ylim()[1]-ax.get_ylim()[0])
    plt.gca().set_aspect(1.2*ratio_default)
    plt.savefig('{}/first_stage_genamount_{}.png'.format(OUT_DIR, r), bbox_inches='tight')
    plt.show()